In [2]:

import numpy as np
import scipy.stats as st
import warnings
import pandas as pd
import numpy_financial as npf
from scipy.stats import lognorm
import scipy.stats 



# <font color='cornflowerblue'> **Tarea 3: Valuation of Investment with distribution**


## **Arantza Gomez Haro Gamboa y Javier Alejandro Fajardo López**

### <font color='cornflowerblue'> **Funciones:**

In [3]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    
    #DISTRIBUTIONS = [st.gennorm,st.genexpon,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,st.nct,st.norm,st.powerlognorm, st.uniform, st.poisson     ]
    DISTRIBUTIONS = [st.norm, st.uniform ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def npv_irr(data, price, prod_cost, op_cost, initial_inv, rf):
    
    def npv(rf, cash_flow):
        return npf.npv(rf, cash_flow)

    def irr(cash_flow):
        return npf.irr(cash_flow) * 100

    NPV = []
    IRR = []

    for i in data.to_numpy():
        c = i * (price - prod_cost) - op_cost

        cash_flow = [-initial_inv]
        cash_flow.extend(c)

        NPV.append(npv(rf, cash_flow))
        IRR.append(irr(cash_flow))

    return NPV, IRR

### <font color='cornflowerblue'> **Datos:**

In [4]:
df = pd.read_csv('Data_OilCompany.csv')
df.columns = ['year1', 'year2', 'year3', 'year4', 'year5']


### <font color='cornflowerblue'> **Pregunta 1:**

In [5]:
df_T = df.T
results_1=[]
for i in df_T.to_numpy():
    distribution = best_fit_distribution(i)
    results_1.append(distribution)
results_1

[('norm', (196593.614, 98549.90341700494)),
 ('uniform', (150096.0, 199816.0)),
 ('uniform', (1.0, 499991.0)),
 ('norm', (199999.3025, 394.1716808622228)),
 ('norm', (72628.2285, 76006.47007692362))]

In [6]:
pregunta_1 = pd.DataFrame({
    'Year 1': 196593.614,
    'Year 2': (150096.0 + 349912)/2
}, index =['Expected PS'])
pregunta_1

,Year 1,Year 2
Expected PS,196593.614,250004.0


### <font color='cornflowerblue'> **Pregunta 2:**

In [7]:
rf = 0.03
price = 5
prod_cost = 0.80
initial_inv = 1800000
op_cost = 40000

In [8]:
npvs_irrs = npv_irr(df, price, prod_cost, op_cost, initial_inv, rf)

In [9]:
results_2=[]
for i in npvs_irrs:
    distribution = best_fit_distribution(i)
    results_2.append(distribution)
results_2

[('norm', (1754930.8721101417, 871671.7965944151)),
 ('norm', (35.92574471411841, 15.477616887962363))]

In [10]:
pregunta_2 = pd.DataFrame({
    'NPV': [1754930.87,(871671.79)**2],
    'IRR': [35.92,(15.47)**2]
}, index =['EV', 'Var'])
pregunta_2

,NPV,IRR
EV,1.754931e+06,35.9200
Var,7.598117e+11,239.3209


### <font color='cornflowerblue'> **Pregunta 3:**

#### **Esperanza del NPV**

Para obtener la esperanza del NPV, nos basaremos en la fórmula del NPV para los 5 años:

$$
NPV = \sum_{t=1}^{5} \frac{CF_t}{(1+r)^t} - \text{Inversión inicial}
$$

donde $ CF_t $ se entiende como el cash flow.

$$
CF_t = \text{Unidades}_t \cdot (\text{Precio} - \text{Costo Producción}) - \text{Costo Operacional}
$$

Donde la única variable que cambia con el tiempo son las unidades de venta.

Tras haber ajustado las unidades de venta por cada año de las 2,000 empresas a la distribución que mejor quedara, y tener conocimiento sobre esperanzas de variable continuas y discretas, la fórmula queda de la siguiente manera:

$$
{E}[NPV] = \sum_{t=1}^{5} \frac{{E}[\text{Unidades}_t] \cdot ({E}[\text{Precio}] - {E}[\text{Costo Producción}] ) - {E}[\text{Costo Operacional}]}{(1+r)^t} - {E}[\text{Inversión inicial}]
$$

Como la única variable aleatoria son las unidades de venta, todas las otras esperanzas se quedan con el valor fijo, y el valor de la variable aleatoria se determinará por la media de la distribución que mejor se acomodó a los datos.

En caso de que la distribución sea normal, la media será el primer valor de los parámetros dados. Si es uniforme, se tomarán los dos parámetros dados y utilizando la función:

$$
\mu = \frac{b + a}{2}
$$

obtenemos la media.

Con esto, sustituimos los valores en la sumatoria y obtenemos que el resultado es:

$$
{E}[NPV] =
\frac{196,593.614 \cdot (4.2) - 40,000}{(1.03)^1} +
\frac{250,004 \cdot (4.2) - 40,000}{(1.03)^2} +
\frac{249,996.5 \cdot (4.2) - 40,000}{(1.03)^3} +
\frac{199,999.302 \cdot (4.2) - 40,000}{(1.03)^4} +
\frac{72,628.228 \cdot (4.2) - 40,000}{(1.03)^5} -
1,800,000
$$

**Resultado Final:**

$$
{E}[NPV] = 1,778,537.863
$$

<font color='cornflowerblue'>

--- 

#### **Varianza del NPV**

Para la varianza, dado que tomaremos los años como independientes entre sí, la fórmula sería:

$$
\text{Var}[NPV] = \sum_{t=1}^{5} \frac{\text{Var}(CF_t)}{(1+r)^{t}} - \text{Var}(\text{Inversión inicial})
$$

Pero como la varianza de una variable constante es 0, la varianza de inversión inicial se elimina, quedando:

$$
\text{Var}[NPV] = \sum_{t=1}^{5} \frac{\text{Var}(CF_t)}{(1+r)^{t}}
$$

Si expandemos la varianza del cash flow en la sumatoria, tenemos:

$$
\text{Var}[NPV] = \sum_{t=1}^{5} \frac{\text{Var}(\text{Unidades}_t \cdot (\text{Precio} - \text{Costo Producción})) - \text{Var}(\text{Costo Operacional})}{(1+r)^{t}}
$$

De igual forma, el costo operacional es una constante, por lo que su varianza será 0.

Aprovechando la propiedad de la varianza de una variable aleatoria por una constante:

$$
\text{Var}(aX) = a^2 \cdot \text{Var}(X)
$$

Podemos reescribir la fórmula de la siguiente manera:

$$
\text{Var}[NPV] = \sum_{t=1}^{5} \left( \frac{\text{Precio} - \text{Costo Producción}}{(1+r)^{t}} \right)^2 \cdot \text{Var}(\text{Unidades}_t)
$$


En caso de que la distribución sea normal, la varianza será el segundo valor de los parámetros dados al cuadrado. Si es uniforme, se tomarán los dos parámetros dados y utilizando la función:

$$
\sigma^2 = \left( \frac{{(b-a)^2}}{12} \right)$$

obtenemos la varianza.

Con esto, sustituimos los valores en la sumatoria y obtenemos que el resultado es:

$$
\text{Var}[NPV] = 
\left[ \left( \frac{4.2}{1.03} \right)^2 \cdot 9,712,083,464.50 \right] + 
\left[ \left( \frac{4.2}{(1.03)^2} \right)^2 \cdot 3,327,202,821 \right] + 
\left[ \left( \frac{4.2}{(1.03)^3} \right)^2 \cdot 20,832,583,340 \right] + 
\left[ \left( \frac{4.2}{(1.03)^4} \right)^2 \cdot 155,371.314 \right] + 
\left[ \left( \frac{4.2}{(1.03)^5} \right)^2 \cdot 5,776,983,494.55 \right]
$$


**Resultado Final:**

$$
Var[NPV] = 597,227,747,012.48
$$

<font color='cornflowerblue'>

--- 

### <font color='cornflowerblue'> **Pregunta 4:**

**1. $P(X > 3)$**

$$
P\left(z > \frac{3 - 35.92}{15.47} \right) = P(z > -2.12) = 0.9829
$$

**Probabilidad: 98.29\%**

<font color='cornflowerblue'>

---


**2. $ P(X > 0.35) $**

$$
P\left(z > \frac{35 - 35.92}{15.47} \right) = P(z > -0.059) = 0.5235
$$

**Probabilidad: 52.35\%**

<font color='cornflowerblue'>

---

**3. $ P(X > 2,000,000) $**

$$
P\left(z > \frac{2,000,000 - 1,754,930.87}{871,671.79} \right) = P(z > 0.28) = 0.3897
$$

**Probabilidad: 38.97%**

<font color='cornflowerblue'>

---

**4. $ P(10 < X < 20) $**

$$
P\left(z > \frac{10 - 35.92}{15.47} \right) = P(z > -1.67)
$$

$$
P\left(z < \frac{20 - 35.92}{15.47} \right) = P(z < -1.02)
$$

$$
P(-1.67 < z < -1.02) = 0.1064
$$

**Probabilidad: 10.64%**

**5. Do you get same responses as in Homework 1.1?**

Tras haber sacado las probabilidades con las PDF y las probabilidades determinísticas, vemos que son muy parecidas entre sí, solamente existe un poco de varianza. 

<font color='cornflowerblue'>